In [5]:
import nbimporter
from gym_env import BlackjackEnvironment
import numpy as np


In [6]:
env = BlackjackEnvironment()

In [13]:
class QLearningAgent:
    def __init__(self, nombre_etats, nombre_actions, learning_rate, discount_factor, epsilon, min_epsilon, epsilon_decay):
        self.nombre_etats = nombre_etats
        self.nombre_actions = nombre_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.min_epsilon = min_epsilon
        self.epsilon_decay = epsilon_decay
        self.Q_table = np.zeros((nombre_etats, nombre_actions))

    def choose_action(self, etat):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.nombre_actions)
        else:
            return np.argmax(self.Q_table[etat])

    def update(self, etat, action, recompense, nouvel_etat):
        # Utiliser uniquement le total de la main du joueur comme état
        etat_index = etat[0]
        nouvel_etat_index = nouvel_etat[0]
        meilleure_future_valeur_Q = np.max(self.Q_table[nouvel_etat_index])
        ancienne_valeur_Q = self.Q_table[etat_index, action]
        nouvelle_valeur_Q = (ancienne_valeur_Q + self.learning_rate *
            (recompense + self.discount_factor * meilleure_future_valeur_Q - ancienne_valeur_Q))
        self.Q_table[etat_index, action] = nouvelle_valeur_Q


    def update_epsilon(self):
        self.epsilon = max(self.min_epsilon, self.epsilon * self.epsilon_decay)


In [15]:
# Paramètres pour l'agent Q-learning
nombre_etats = 36  # Adaptez selon votre environnement
nombre_actions = 2  # Stand ou Hit
learning_rate = 0.1
discount_factor = 0.99
epsilon = 1.0
min_epsilon = 0.01
epsilon_decay = 0.995

# Création de l'agent Q-learning
agent = QLearningAgent(nombre_etats, nombre_actions, learning_rate, discount_factor, epsilon, min_epsilon, epsilon_decay)

# Boucle d'apprentissage
nombre_episodes = 10000
for episode in range(nombre_episodes):
    etat_initial = env.reset_environment()
    etat = etat_initial

    fini = False

    while not fini:
        action = agent.choose_action(etat)
        resultat = env.step(action)
        
        nouvel_etat, recompense, fini, _, _ = resultat
        agent.update(etat, action, recompense, nouvel_etat)
        etat = nouvel_etat

